In [6]:
import os 
import pandas as pd
import numpy as np
import subprocess
import glob
import pybedtools as pbt 
from IPython.display import HTML
pd.set_option('display.max_rows', 200) 
pd.set_option('display.max_columns', None)
import upsetplot

import seaborn as sns 
import matplotlib.pyplot as plt 

pbt.set_bedtools_path('/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/')
#pbt.set_tempdir('/mnt/hpcscratch/jreyna/')
os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/t1d-loop-catalog/')

gsizes = 'results/hg38/refs/hg38/hg38.chrom.sizes'
res = 5000

# make the directory to save our data
outdir = 'results/hg38/finemapping/sgl_intersect/'
os.makedirs(outdir, exist_ok=True)
bedpe_cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB']

## Load Fine Mapped GWAS

In [11]:
fn = 'results/hg38/external_studies/chiou_2021/processing/finemapping/hg38.finemapping.basic.bed'
# get the bin coordinates
gwas_df = pd.read_table(fn)
# df.loc[:, 'bin_start'] = np.floor(df.loc[:, 'position'] / res).astype(int) * res
# df.loc[:, 'bin_end'] = df.loc[:, 'bin_start'] + res
# df = df.loc[(df.allele1.str.len() == 1 ) & (df.allele2.str.len() == 1)]

In [12]:
gwas_df.shape

(46738, 5)

In [8]:
# filter for PIP > 0.05
gwas_df = gwas_df.loc[gwas_df.ppa > 0.05]

In [9]:
gwas_df.head()

,chrom,start,end,rsid,ppa
105,1,35571620,35571621,rs6425948,0.130213
112,1,35622059,35622060,rs574384,0.195293
115,1,35641908,35641909,rs676614,0.186331
117,1,35666944,35666945,rs596471,0.084043
236,1,37879348,37879349,rs4585948,0.064023


In [10]:
gwas_df.shape

(490, 5)

In [5]:
# create a pybedtools for finemap data
gwas_pbt = pbt.BedTool.from_dataframe(gwas_df)

## Load HiChIP Loops

In [6]:
# only analyze loop data from main cell types 
loop_fn = 'results/hg38/loops/hichip/chip-seq/macs2/loose/Jurkat.GSE99519.Homo_Sapiens.YY1.b1.25000.interactions_FitHiC_Q0.01.bed'
loop_df = pd.read_table(loop_fn)
loop_df['-log10_qval'] =  loop_df['Q-Value_Bias'].apply(lambda x: -np.log(x))
loop_df['chr1'] = loop_df['chr1'].str.replace('chr', '') 
loop_df['chr2'] = loop_df['chr2'].str.replace('chr', '') 

In [7]:
print(loop_df.shape)

(19844, 27)


In [8]:
# create a dataframe in bed format which filters for significant
# SNPs only p-val < 0.05 (or -log10(p-val) > 1.3)
loop_bed = loop_df.iloc[:, [0,1,2,3,4,5,-1]]

# create a pybedtools for the looping data
loop_pbt = pbt.BedTool.from_dataframe(loop_bed)

In [9]:
loop_bed.head()

,chr1,s1,e1,chr2,s2,e2,-log10_qval
0,1,975000,1000000,1,1300000,1325000,7.087591
1,1,1000000,1025000,1,1225000,1250000,6.426370
2,1,1000000,1025000,1,1250000,1275000,19.420019
3,1,1000000,1025000,1,1275000,1300000,7.210015
4,1,1025000,1050000,1,1400000,1425000,6.051496


## Intersect Fine Mapped GWAS and loops

In [10]:
gwas_pbt.to_dataframe()

,chrom,start,end,name,score
0,1,35571620,35571621,rs6425948,0.130213
1,1,35622059,35622060,rs574384,0.195293
2,1,35641908,35641909,rs676614,0.186331
3,1,35666944,35666945,rs596471,0.084043
4,1,37879348,37879349,rs4585948,0.064023
...,...,...,...,...,...
485,9,4283136,4283137,rs1574285,0.213968
486,9,4289049,4289050,rs7034200,0.186268
487,9,4289195,4289196,rs10814914,0.089392
488,9,4292082,4292083,rs10758593,0.069463


#### Perform the intersection

In [11]:
# intersecting the GWAS SNPs with loops 
# loop anchor doesn't matter therefore type='either'
intersect_pbt = loop_pbt.pair_to_bed(gwas_pbt, type='either')

In [12]:
# # convert the intersection into a dataframe and rename columns 
gwas_hichip = intersect_pbt.to_dataframe(header=None, disable_auto_names=True)

if len(gwas_hichip) == 0:
    print('No overlap between GWAS and HiChIP loops')
    
gwas_hichip.columns = ['chrA_loop', 'startA_loop', 'endA_loop',
                       'chrB_loop', 'startB_loop', 'endB_loop', 
                       '-log10_qval', 'chr_snp', 'start_snp', 'end_snp', 'rsid', 'ppa_snp']

# gwas_hichip = gwas_hichip.iloc[:, [7,8,9,10,0,1,2,3,4,5,6,11]]
# loop_cols = ['{}_loop'.format(x) for x in bedpe_cols]
# gwas_hichip.columns = ['chr_snp', 'bin_start', 'bin_end', 'pos'] + loop_cols + ['cline_loop', 'gwas_source']

# # add back fields from the original gwas data
# # some data was lost in the pybedtool process
# gwas_hichip = gwas_hichip.merge(gwas_df.drop('gwas_source', axis=1),
#                                 left_on=['chr_snp', 'pos'],
#                                 right_on=['chromosome', 'position'])
# # add the sid as the variant id
# gwas_hichip['sid'] = 'chr' +  gwas_hichip['chr_snp'].astype(str) + ':' + gwas_hichip['position'].astype(str)

## Integrate genes and located SGLs

For the types of SGLs through this approach we have to find identify loops where one anchor overlaps a SNP and the other one overlaps a gene promoter. As such, we used the previously intersected data to find loop anchors without a SNP overlap and we will intersect these anchors with gene promoters next. 

### Load the gene data and build promoter based pybedtool

In [13]:
print('# Load the gene data')

genes_fn = 'results/hg38/refs/gencode/v30/gencode.v30.annotation.bed'

# load the gencode coords
cols = ['chrom', 'start', 'end', 'strand', 'type', 'gene_id', 'gname']
gencode = pd.read_table(genes_fn, header=None, names=cols)

# extract just the genes
genes_df = gencode.loc[gencode['type'].isin(['gene'])]
genes_df = genes_df.loc[~genes_df.duplicated(subset='gene_id'), :]
genes_df.loc[:, 'chrom'] = genes_df['chrom'].astype(str)
genes_df = genes_df.iloc[:, [0,1,2,6,5,3]]

# create a copy of the original gene bed before coordinate shrinking
orig_genes_df = genes_df.copy()

# convert the start/end position into start/end for the TSS
# if the gene is + then the start is uses as the tss otherwise
# the end is used as the tss
genes_df.loc[(genes_df.strand == '+'), 'end'] = genes_df.loc[(genes_df.strand == '+'), 'start']
genes_df.loc[(genes_df.strand == '+'), 'start'] = genes_df.loc[(genes_df.strand == '+'), 'start'] - 1
genes_df.loc[(genes_df.strand == '-'), 'end'] = genes_df.loc[(genes_df.strand == '-'), 'end']
genes_df.loc[(genes_df.strand == '-'), 'start'] = genes_df.loc[(genes_df.strand == '-'), 'end'] - 1
genes_df.loc[:, 'chrom'] = genes_df.loc[:, 'chrom'].str.replace('chr', '')
genes_df.loc[:, 'bin_start'] = (np.floor(genes_df.loc[:, 'start'] / res) * res).astype(int)
genes_df.loc[:, 'bin_end'] = genes_df.loc[:, 'bin_start'] + res

# make a genes pbt for intersection
print("# make a genes pbt for intersection")
print(genes_df.head())
genes_pbt = pbt.BedTool.from_dataframe(genes_df).sort()

print('There are {} genes in this GTF-derived file.'.format(genes_df.shape[0]))

# Load the gene data


: 

: 

### Determine which anchor the SNP falls into

In [ ]:
snp_anchor = []
for i, sr in gwas_hichip.iterrows():
    if (sr.startA_loop <= sr.end_snp) & (sr.end_snp <= sr.endA_loop):
        snp_anchor.append('AnchorA')
    elif (sr.startB_loop <= sr.end_snp) & (sr.end_snp <= sr.endB_loop):
        snp_anchor.append('AnchorB')
    else:
        snp_anchor.append('bug')
        print('bug')
        break
gwas_hichip.loc[:, 'snp_anchor'] = snp_anchor

In [ ]:
print('SNP anchor designation:', gwas_hichip['snp_anchor'].unique().tolist())

SNP anchor designation: ['AnchorB', 'AnchorA']


### Extract anchors opposite of a SNP anchor

In [ ]:
# using a basic serial id for merging post bedtools intersection
gwas_hichip['gh_id'] = range(gwas_hichip.shape[0])

anchor_cols = ['chrB_loop', 'startB_loop', 'endB_loop', 'gh_id']
nonsnp_anchorsA = gwas_hichip.loc[gwas_hichip['snp_anchor'] == 'AnchorA', anchor_cols]
anchor_cols =  ['chrA_loop', 'startA_loop', 'endA_loop', 'gh_id']
nonsnp_anchorsB = gwas_hichip.loc[gwas_hichip['snp_anchor'] == 'AnchorB', anchor_cols]

nonsnp_anchorsA.columns = ['chr', 'start', 'end', 'gh_id']
nonsnp_anchorsB.columns = ['chr', 'start', 'end', 'gh_id']
nonsnp_anchors = pd.concat([nonsnp_anchorsA, nonsnp_anchorsB], axis=0)
nonsnp_anchors_pbt = pbt.BedTool.from_dataframe(nonsnp_anchors)

In [ ]:
nonsnp_anchors.head()

,chr,start,end,gh_id
2,1,35625000,35650000,2
4,1,35700000,35725000,4
5,1,35725000,35750000,5
6,1,35750000,35775000,6
7,1,35775000,35800000,7


### Intersecting genes on anchors opposing a SNP anchor

In [ ]:
gene_overlaps = nonsnp_anchors_pbt.intersect(genes_pbt, wa=True, wb=True)
gene_overlaps = gene_overlaps.to_dataframe(header=None, disable_auto_names=True)

In [ ]:
print('The number of anchor gene overlaps is:', gene_overlaps.shape)

The number of anchor gene overlaps is: (798, 12)


In [ ]:
gene_overlaps.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,35625000,35650000,2,1,35641525,35641526,PSMB2,ENSG00000126067,-,35640000,35645000
1,1,35700000,35725000,4,1,35718893,35718894,C1orf216,ENSG00000142686,-,35715000,35720000
2,1,35725000,35750000,5,1,35739388,35739389,AL354864.1,ENSG00000232335,+,35735000,35740000
3,1,35750000,35775000,6,1,35769977,35769978,CLSPN,ENSG00000092853,-,35765000,35770000
4,1,37975000,38000000,8,1,37990074,37990075,SF3A3,ENSG00000183431,-,37990000,37995000


### Add gene overlaps to SNP-Loop Pairs

In [ ]:
gene_overlaps.columns = ['chr_anchor', 'start_anchor', 'end_anchor', 'gh_id',
                         'chr_gene', 'start_gene', 'end_gene',
                         'genename', 'geneid', 'strand', 'bin_start', 'bin_end']
gwas_hichip_genes = gwas_hichip.merge(gene_overlaps.drop(['', '', ''], axis=1), on=['gh_id'], how='inner')

In [ ]:
print('There are {} SGLs.'.format(len(gwas_hichip_genes)))

There are 798 SGLs.


In [ ]:
print('There are {} unique SNPs within an SGL'.format(gwas_hichip_genes[['rsid']].nunique()))

There are rsid    184
dtype: int64 unique SNPs within an SGL


In [ ]:
gwas_hichip_genes

,chrA_loop,startA_loop,endA_loop,chrB_loop,startB_loop,endB_loop,-log10_qval,chr_snp,start_snp,end_snp,rsid,ppa_snp,snp_anchor,gh_id,chr_anchor,start_anchor,end_anchor,chr_gene,start_gene,end_gene,genename,geneid,strand,bin_start,bin_end
0,1,35550000,35575000,1,35625000,35650000,4.919412,1,35571620,35571621,rs6425948,0.130213,AnchorA,2,1,35625000,35650000,1,35641525,35641526,PSMB2,ENSG00000126067,-,35640000,35645000
1,1,35550000,35575000,1,35625000,35650000,4.919412,1,35641908,35641909,rs676614,0.186331,AnchorB,3,1,35550000,35575000,1,35557472,35557473,NCDN,ENSG00000020129,+,35555000,35560000
2,1,35550000,35575000,1,35625000,35650000,4.919412,1,35641908,35641909,rs676614,0.186331,AnchorB,3,1,35550000,35575000,1,35557949,35557950,KIAA0319L,ENSG00000142687,-,35555000,35560000
3,1,35550000,35575000,1,35625000,35650000,4.919412,1,35641908,35641909,rs676614,0.186331,AnchorB,3,1,35550000,35575000,1,35573313,35573314,TFAP2E,ENSG00000116819,+,35570000,35575000
4,1,35550000,35575000,1,35700000,35725000,38.252608,1,35571620,35571621,rs6425948,0.130213,AnchorA,4,1,35700000,35725000,1,35718893,35718894,C1orf216,ENSG00000142686,-,35715000,35720000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,8,140500000,140525000,8,140600000,140625000,8.287003,8,140608622,140608623,rs884934,0.086075,AnchorB,661,8,140500000,140525000,8,140511310,140511311,CHRAC1,ENSG00000104472,+,140510000,140515000
794,8,140500000,140525000,8,140600000,140625000,8.287003,8,140612790,140612791,rs6998901,0.063517,AnchorB,662,8,140500000,140525000,8,140508042,140508043,AC107375.1,ENSG00000259891,-,140505000,140510000
795,8,140500000,140525000,8,140600000,140625000,8.287003,8,140612790,140612791,rs6998901,0.063517,AnchorB,662,8,140500000,140525000,8,140511310,140511311,CHRAC1,ENSG00000104472,+,140510000,140515000
796,8,140500000,140525000,8,140625000,140650000,14.203714,8,140630634,140630635,rs13248602,0.056641,AnchorB,663,8,140500000,140525000,8,140508042,140508043,AC107375.1,ENSG00000259891,-,140505000,140510000


## Save SGLs for Further Analyses

In [ ]:
finemap_sgls_fn = os.path.join(outdir, 'finemap_sgls.tsv')
gwas_hichip_genes.to_csv(finemap_sgls_fn,  sep='\t', index=False)

In [ ]:
gwas_hichip_genes

,chrA_loop,startA_loop,endA_loop,chrB_loop,startB_loop,endB_loop,-log10_qval,chr_snp,start_snp,end_snp,rsid,ppa_snp,snp_anchor,gh_id,chrSNP,startSNP,endSNP,chrGene,startGene,endGene,genename,geneid,strand,bin_start,bin_end
2,1,35550000,35575000,1,35625000,35650000,4.919412,1,35571620,35571621,rs6425948,0.130213,AnchorA,2,1.0,35625000.0,35650000.0,1.0,35641525.0,35641526.0,PSMB2,ENSG00000126067,-,35640000.0,35645000.0
3,1,35550000,35575000,1,35625000,35650000,4.919412,1,35641908,35641909,rs676614,0.186331,AnchorB,3,1.0,35550000.0,35575000.0,1.0,35557472.0,35557473.0,NCDN,ENSG00000020129,+,35555000.0,35560000.0
4,1,35550000,35575000,1,35625000,35650000,4.919412,1,35641908,35641909,rs676614,0.186331,AnchorB,3,1.0,35550000.0,35575000.0,1.0,35557949.0,35557950.0,KIAA0319L,ENSG00000142687,-,35555000.0,35560000.0
5,1,35550000,35575000,1,35625000,35650000,4.919412,1,35641908,35641909,rs676614,0.186331,AnchorB,3,1.0,35550000.0,35575000.0,1.0,35573313.0,35573314.0,TFAP2E,ENSG00000116819,+,35570000.0,35575000.0
6,1,35550000,35575000,1,35700000,35725000,38.252608,1,35571620,35571621,rs6425948,0.130213,AnchorA,4,1.0,35700000.0,35725000.0,1.0,35718893.0,35718894.0,C1orf216,ENSG00000142686,-,35715000.0,35720000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,8,140500000,140525000,8,140600000,140625000,8.287003,8,140608622,140608623,rs884934,0.086075,AnchorB,661,8.0,140500000.0,140525000.0,8.0,140511310.0,140511311.0,CHRAC1,ENSG00000104472,+,140510000.0,140515000.0
1098,8,140500000,140525000,8,140600000,140625000,8.287003,8,140612790,140612791,rs6998901,0.063517,AnchorB,662,8.0,140500000.0,140525000.0,8.0,140508042.0,140508043.0,AC107375.1,ENSG00000259891,-,140505000.0,140510000.0
1099,8,140500000,140525000,8,140600000,140625000,8.287003,8,140612790,140612791,rs6998901,0.063517,AnchorB,662,8.0,140500000.0,140525000.0,8.0,140511310.0,140511311.0,CHRAC1,ENSG00000104472,+,140510000.0,140515000.0
1100,8,140500000,140525000,8,140625000,140650000,14.203714,8,140630634,140630635,rs13248602,0.056641,AnchorB,663,8.0,140500000.0,140525000.0,8.0,140508042.0,140508043.0,AC107375.1,ENSG00000259891,-,140505000.0,140510000.0
